# Geographical location of accounts

* Merge the "Goldmine Territory and 2018 Trade Shows.xlsx" account territory with the accounts in the yaerly dataset (take 2024 sales dataset for example)
* Examine whether all the accounts have a geagraphic location recorded in Client"s datasets

In [1]:
import pandas as pd
import numpy as np

## Import and Examine the Account Location File

* Examine the total number of accounts (companies) recorded in the xlsx file.
* Examine the number of accounts that has a geographic territory recorded in the dataset
* create a composite key for each account

In [3]:
geo = pd.read_excel("Goldmine Territory and 2018 Trade Shows.xlsx")
print(geo.head())

        C1AccountNo        CXRecords Residence  Cmpy Sales Proj 2012  \
0  A2061733055&RY+5  9147BAL$5?9W W<       NaN                   NaN   
1  A2061733056$=L._  9S575WH#XX 1 YU       NaN                   NaN   
2  A2061733058#?\R?  CENFZ26$1HH03{G       NaN                   NaN   
3  A2061733058#^F7+  91470IF(9`JJ W<       NaN                   NaN   
4  A2061733059#G9O;  91470JW%5D/N W<       NaN                   NaN   

   Comp Terr   Latitude  UPRJISLE11 Record Confirm  UREORDCALL      Rebate  \
0    Central   0.000000         NaN     1989-01-01         NaN         NaN   
1    Western  37.773032         NaN     1989-01-01         NaN         NaN   
2    Midwest  44.048492         NaN     1989-01-01         NaN         NaN   
3    Central  41.579921         NaN     1989-01-01         NaN         NaN   
4  Northeast  27.984706         NaN     1989-01-01         NaN  4%>$25,000   

  Trd Shw 2018 Facebook Link  
0          NaN           NaN  
1          NaN           NaN  
2    

In [5]:
#Examine the missing values in each column
# Check the number of missing values in each column
missing_values = geo.isnull().sum()

print(len(geo))
print(missing_values)

124414
C1AccountNo                  0
CXRecords                    0
Residence               106473
Cmpy Sales Proj 2012    124152
Comp Terr                 6559
Latitude                     0
UPRJISLE11              124414
Record Confirm               0
UREORDCALL              124281
Rebate                  100567
Trd Shw 2018            123043
Facebook Link           113514
dtype: int64


In [6]:
#The above result shows that there are comapny without Terro value but with Latitude value
#Examine the rows that satisfy this situation
# Filter rows where 'Comp Terr' is NaN but 'Latitude' is not NaN
missing_comp_terr_with_latitude = geo[geo['Comp Terr'].isna() & geo['Latitude'].notna()]

print(missing_comp_terr_with_latitude.head())
print(missing_comp_terr_with_latitude.iloc[10])


          C1AccountNo        CXRecords Residence  Cmpy Sales Proj 2012  \
73   A2061733056(X^F!  91470FL)3O[F W<       NaN                   NaN   
120  A2061733081#A_?%  91471EC!WY*C W<       NaN                   NaN   
190  A2061733082(QC5#  91471FM!.M_\ W<       NaN                   NaN   
204  A2061733091$,4;U  91471RX%9(R( W<       NaN                   NaN   
217  A2061733060!-W P  91470L2#:#8F W<       NaN                   NaN   

    Comp Terr   Latitude  UPRJISLE11 Record Confirm  UREORDCALL Rebate  \
73        NaN  41.698611         NaN     1989-01-01         NaN    NaN   
120       NaN   0.000000         NaN     1989-01-01         NaN      Y   
190       NaN   0.000000         NaN     1989-01-01         NaN    NaN   
204       NaN  41.698958         NaN     1989-01-01         NaN    NaN   
217       NaN   0.000000         NaN     1989-01-01         NaN    NaN   

    Trd Shw 2018 Facebook Link  
73           NaN           NaN  
120          NaN           NaN  
190        

In [7]:
# while some attitude is 0.00 (no value in this case), some latitude has a non 0 value but without Company Terr valure
#filter out those rows and assign similar company Terr value to those rows

#This is for testing the matching code

geo_copy = geo.copy(deep=True)

#Filter rows with valid Latitude (> 0.00) and Comp Terr values as reference examples
reference_geo = geo_copy[(geo_copy['Latitude'] > 0.00) & geo_copy['Comp Terr'].notna()]

#Filter rows that have a valid Latitude (> 0.00) but missing Comp Terr
missing_comp_terr_rows = geo_copy[(geo_copy['Latitude'] > 0.00) & geo_copy['Comp Terr'].isna()]

#Function to find the closest latitude and its corresponding Comp Terr
def find_closest_latitude(row, reference_df):
    latitude = row['Latitude']
    # Find the row in the reference data with the closest latitude
    closest_row = reference_df.iloc[(reference_df['Latitude'] - latitude).abs().argsort()[:1]]
    return closest_row['Comp Terr'].values[0]  # Return the closest Comp Terr value

#Assign the closest Comp Terr value to rows with missing Comp Terr in the dataframe
geo_copy.loc[missing_comp_terr_rows.index, 'Comp Terr'] = missing_comp_terr_rows.apply(find_closest_latitude, reference_df=reference_geo, axis=1)

missing_values_test = geo_copy.isnull().sum()

missing_comp_terr_with_latitude_test = geo_copy[geo_copy['Comp Terr'].isna() & geo_copy['Latitude'].notna()]

print(geo_copy.head())
print(missing_comp_terr_with_latitude_test.head())
print(len(missing_comp_terr_with_latitude_test))


C:\Users\Michael\AppData\Local\Temp\ipykernel_68084\2166577889.py:18: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  closest_row = reference_df.iloc[(reference_df['Latitude'] - latitude).abs().argsort()[:1]]


        C1AccountNo        CXRecords Residence  Cmpy Sales Proj 2012  \
0  A2061733055&RY+5  9147BAL$5?9W W<       NaN                   NaN   
1  A2061733056$=L._  9S575WH#XX 1 YU       NaN                   NaN   
2  A2061733058#?\R?  CENFZ26$1HH03{G       NaN                   NaN   
3  A2061733058#^F7+  91470IF(9`JJ W<       NaN                   NaN   
4  A2061733059#G9O;  91470JW%5D/N W<       NaN                   NaN   

   Comp Terr   Latitude  UPRJISLE11 Record Confirm  UREORDCALL      Rebate  \
0    Central   0.000000         NaN     1989-01-01         NaN         NaN   
1    Western  37.773032         NaN     1989-01-01         NaN         NaN   
2    Midwest  44.048492         NaN     1989-01-01         NaN         NaN   
3    Central  41.579921         NaN     1989-01-01         NaN         NaN   
4  Northeast  27.984706         NaN     1989-01-01         NaN  4%>$25,000   

  Trd Shw 2018 Facebook Link  
0          NaN           NaN  
1          NaN           NaN  
2    

In [9]:
print(len(missing_comp_terr_with_latitude_test))
print(missing_values_test)

5053
C1AccountNo                  0
CXRecords                    0
Residence               106473
Cmpy Sales Proj 2012    124152
Comp Terr                 5053
Latitude                     0
UPRJISLE11              124414
Record Confirm               0
UREORDCALL              124281
Rebate                  100567
Trd Shw 2018            123043
Facebook Link           113514
dtype: int64


### Summary stat for Account Geo Location Info
* Currently there are 6559 accounts inside thw dataset without geographical location (Company Terr is NA and Latitude == 0.000 or Latitude has a value other than 0)
* If we use the existing Company terr and Latitude to impute the rows that as a non 0 Latitude and NA Company terr, there will be 5053 rows left that we can not determine the location of use a close latitude for imputation
* Consider that the missing value is less than 6% of the entire dataset, we could first merge the dataset for analysis.

## Merge the Company Location value with the annual sales dataset

* create composite key for the geo dataframe
* examine whether the same key exists in the 2023 sales and 2024 sales dataset
* Merge the 2023 sales dataset
* Merge the 2024 sales dataset

In [14]:
# Create a composite key by combining 'C1AccountNo' and 'CXRecords' columns
geo['composite_key'] = geo['C1AccountNo'] + '-' + geo['CXRecords']


print(geo[['C1AccountNo', 'CXRecords', 'composite_key']].head())

        C1AccountNo        CXRecords                     composite_key
0  A2061733055&RY+5  9147BAL$5?9W W<  A2061733055&RY+5-9147BAL$5?9W W<
1  A2061733056$=L._  9S575WH#XX 1 YU  A2061733056$=L._-9S575WH#XX 1 YU
2  A2061733058#?\R?  CENFZ26$1HH03{G  A2061733058#?\R?-CENFZ26$1HH03{G
3  A2061733058#^F7+  91470IF(9`JJ W<  A2061733058#^F7+-91470IF(9`JJ W<
4  A2061733059#G9O;  91470JW%5D/N W<  A2061733059#G9O;-91470JW%5D/N W<


In [15]:
#read in the 2023 and 2024 sales datasets

sales2023 = pd.read_excel('Crystal D 2023 sales.xlsx')
sales2024 = pd.read_excel('Crystal D 2024 sales.xlsx')

print(sales2023.shape)
print(sales2024.shape)

(124414, 20)
(124414, 18)


In [16]:
print(sales2023.head())


        C1AccountNo        CXRecords                     composite_key  \
0  A2061733055&RY+5  91470ER(#7ZS W<  A2061733055&RY+5-91470ER(#7ZS W<   
1  A2061733055)S{G]  91470F3(E^Q/ W<  A2061733055)S{G]-91470F3(E^Q/ W<   
2  A2061733056$FX$L  91470G4$R)&< W<  A2061733056$FX$L-91470G4$R)&< W<   
3  A2061733057#*N-2  91470GQ%![I! W<  A2061733057#*N-2-91470GQ%![I! W<   
4  A2061733058#^F7+  91470IF(9`JJ W<  A2061733058#^F7+-91470IF(9`JJ W<   

   2023 Jan  2023 Feb  2023 Mar  2023 Apr  2023 May  2023 Jun  2023 Jul  \
0      0.00      0.00      0.00      0.00      0.00       0.0       0.0   
1      0.00      0.00      0.00      0.00      0.00       0.0       0.0   
2      0.00      0.00      0.00      0.00      0.00       0.0       0.0   
3  14960.72   1582.26   2425.43   4863.82  13153.96       0.0       0.0   
4      0.00      0.00      0.00      0.00      0.00       0.0       0.0   

   2023 Aug  2023 Sep  2023 Oct  2023 Nov  2023 Dec   2023 Q1   2023 Q2  \
0       0.0       0.0       0

In [17]:
#Examine whether the composite key exists in both datasets
# Convert the 'composite_key' columns of both dataframes to sets
geo_keys = set(geo['composite_key'])
sales_keys = set(sales2023['composite_key'])

keys_in_geo_not_in_sales = geo_keys - sales_keys
keys_in_sales_not_in_geo = sales_keys - geo_keys

print("Composite keys in geo but not in sales2023:")
print(keys_in_geo_not_in_sales)

print("\nComposite keys in sales2023 but not in geo:")
print(keys_in_sales_not_in_geo)

Composite keys in geo but not in sales2023:
set()

Composite keys in sales2023 but not in geo:
set()


In [20]:
#Since there are no key that is mismatched in the data frames, merge the geo info to the sales 2023 data frame
# Merge the 'Comp Terr' and 'Latitude' columns from geo into sales2023 based on the composite_key
sales2023 = sales2023.merge(geo[['composite_key', 'Comp Terr', 'Latitude']], on='composite_key', how='left')

# Display the result to check if the merge worked correctly
print(sales2023.head())
print((len(sales2023)))

        C1AccountNo        CXRecords                     composite_key  \
0  A2061733055&RY+5  91470ER(#7ZS W<  A2061733055&RY+5-91470ER(#7ZS W<   
1  A2061733055)S{G]  91470F3(E^Q/ W<  A2061733055)S{G]-91470F3(E^Q/ W<   
2  A2061733056$FX$L  91470G4$R)&< W<  A2061733056$FX$L-91470G4$R)&< W<   
3  A2061733057#*N-2  91470GQ%![I! W<  A2061733057#*N-2-91470GQ%![I! W<   
4  A2061733058#^F7+  91470IF(9`JJ W<  A2061733058#^F7+-91470IF(9`JJ W<   

   2023 Jan  2023 Feb  2023 Mar  2023 Apr  2023 May  2023 Jun  2023 Jul  ...  \
0      0.00      0.00      0.00      0.00      0.00       0.0       0.0  ...   
1      0.00      0.00      0.00      0.00      0.00       0.0       0.0  ...   
2      0.00      0.00      0.00      0.00      0.00       0.0       0.0  ...   
3  14960.72   1582.26   2425.43   4863.82  13153.96       0.0       0.0  ...   
4      0.00      0.00      0.00      0.00      0.00       0.0       0.0  ...   

   2023 Oct  2023 Nov  2023 Dec   2023 Q1   2023 Q2  2023 Q3  2023 Q4  \
0

In [21]:
#Examine the rows in the sales2023 which the Company Terr is NA and Latitude is not NA, check whether the number of rows is 6559
rows_with_latitude_no_comp_terr = sales2023[(sales2023['Latitude'].notna()) & (sales2023['Comp Terr'].isna())]

print(len(rows_with_latitude_no_comp_terr))

6559


In [23]:
print(sales2024.head())

        C1AccountNo        CXRecords                     composite_key  \
0  A2061733055&RY+5  91470ER(#7ZS W<  A2061733055&RY+5-91470ER(#7ZS W<   
1  A2061733055)S{G]  91470F3(E^Q/ W<  A2061733055)S{G]-91470F3(E^Q/ W<   
2  A2061733056$FX$L  91470G4$R)&< W<  A2061733056$FX$L-91470G4$R)&< W<   
3  A2061733057#*N-2  91470GQ%![I! W<  A2061733057#*N-2-91470GQ%![I! W<   
4  A2061733058#^F7+  91470IF(9`JJ W<  A2061733058#^F7+-91470IF(9`JJ W<   

   2024 Jan  2024 Feb  2024 Mar  2024 Apr  2024 May  2024 Jun  2024 Jul  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   2024 Aug  2024 Sep  2024 Oct  2024 Nov  2024 Dec  2024 Q1  2024 Q2  \
0       0.0         0         0

In [24]:
#Examine whether the composite key exists in both datasets (sales2024 and geo)
# Convert the 'composite_key' columns of both dataframes to sets
sales_keys_2024 = set(sales2024['composite_key'])

keys_in_geo_not_in_sales_2024 = geo_keys - sales_keys_2024
keys_in_sales_not_in_geo_2024 = sales_keys_2024 - geo_keys

print("Composite keys in geo but not in sales2024:")
print(keys_in_geo_not_in_sales_2024)

print("\nComposite keys in sales2024 but not in geo:")
print(keys_in_sales_not_in_geo_2024)

Composite keys in geo but not in sales2024:
set()

Composite keys in sales2024 but not in geo:
set()


In [25]:
#Since there are no key that is mismatched in the data frames, merge the geo info to the sales 2024 data frame
# Merge the 'Comp Terr' and 'Latitude' columns from geo into sales2024 based on the composite_key
sales2024 = sales2024.merge(geo[['composite_key', 'Comp Terr', 'Latitude']], on='composite_key', how='left')

# Display the result to check if the merge worked correctly
print(sales2024.head())
print((len(sales2024)))

        C1AccountNo        CXRecords                     composite_key  \
0  A2061733055&RY+5  91470ER(#7ZS W<  A2061733055&RY+5-91470ER(#7ZS W<   
1  A2061733055)S{G]  91470F3(E^Q/ W<  A2061733055)S{G]-91470F3(E^Q/ W<   
2  A2061733056$FX$L  91470G4$R)&< W<  A2061733056$FX$L-91470G4$R)&< W<   
3  A2061733057#*N-2  91470GQ%![I! W<  A2061733057#*N-2-91470GQ%![I! W<   
4  A2061733058#^F7+  91470IF(9`JJ W<  A2061733058#^F7+-91470IF(9`JJ W<   

   2024 Jan  2024 Feb  2024 Mar  2024 Apr  2024 May  2024 Jun  2024 Jul  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   2024 Aug  2024 Sep  2024 Oct  2024 Nov  2024 Dec  2024 Q1  2024 Q2  \
0       0.0         0         0

In [26]:
#Examine the rows in the sales2024 which the Company Terr is NA and Latitude is not NA, check whether the number of rows is 6559
rows_with_latitude_no_comp_terr_2024 = sales2024[(sales2024['Latitude'].notna()) & (sales2024['Comp Terr'].isna())]

print(len(rows_with_latitude_no_comp_terr_2024))

6559


## Summary of two datasets and export to excel document

* The keys for these datasets are the same
* there are same number of missing geolocation info rows that needs to be further examined

In [27]:
#Export both dataframes to excel files
file_2023_name = 'Crystal D 2023 sales with geo.xlsx'
file_2024_name = 'Crystal D 2024 sales with geo.xlsx'
sales2023.to_excel(file_2023_name, index=False)
sales2024.to_excel(file_2024_name, index=False)

print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
